In [2]:
%pip install faiss-cpu


   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ------ --------------------------------- 2.4/14.9 MB 10.3 MB/s eta 0:00:02
   ------------- -------------------------- 5.0/14.9 MB 11.2 MB/s eta 0:00:01
   ------------------- -------------------- 7.3/14.9 MB 11.3 MB/s eta 0:00:01
   -------------------------- ------------- 10.0/14.9 MB 11.5 MB/s eta 0:00:01
   --------------------------------- ------ 12.6/14.9 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------  14.7/14.9 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 10.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install pinecone-client sentence-transformers


   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.0 MB 12.2 MB/s eta 0:00:01
   ------------------ --------------------- 5.0/11.0 MB 11.6 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 11.7 MB/s eta 0:00:01
   ------------------------------------ --- 10.0/11.0 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 10.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install fastapi

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install uvicorn

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install pytest httpx


Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install python-multipart

In [2]:
%pip install --upgrade langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install langserve


  Using cached langserve-0.3.0-py3-none-any.whl.metadata (39 kB)
Using cached langserve-0.3.0-py3-none-any.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install sse_starlette

Note: you may need to restart the kernel to use updated packages.
